# Binary Classification Deep Learning Model for [Project Name] Using Keras Version 3
### David Lowe
### November 1, 2019
Template Credit: Adapted from a template made available by Dr. Jason Brownlee of Machine Learning Mastery. [https://machinelearningmastery.com/]

SUMMARY: [Sample Paragraph - The purpose of this project is to construct a predictive model using various machine learning algorithms and to document the end-to-end steps using a template. The Connectionist Bench dataset is a binary classification situation where we are trying to predict one of the two possible outcomes.]

INTRODUCTION: [Sample Paragraph - The data file patterns obtained by bouncing sonar signals off a metal cylinder or a rock at various angles and under various conditions. The transmitted sonar signal is a frequency-modulated chirp, rising in frequency. The data set contains signals obtained from a variety of different aspect angles, spanning 90 degrees for the cylinder and 180 degrees for the rock. Each pattern is a set of 60 numbers in the range 0.0 to 1.0. Each number represents the energy within a particular frequency band, integrated over a certain period of time.]

ANALYSIS: [Sample Paragraph - The baseline performance of the model achieved an average accuracy score of 80.12%. Using the same training parameters, the model processed the test dataset with an accuracy of 82.69%, which was even better than results from the training data.]

CONCLUSION: [Sample Paragraph - For this dataset, the model built using Keras and TensorFlow achieved a satisfactory result and should be considered for future modeling activities]

Dataset Used: [Connectionist Bench (Sonar, Mines vs. Rocks) Data Set]

Dataset ML Model: Binary classification with numerical attributes

Dataset Reference: [https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+%28Sonar,+Mines+vs.+Rocks%29]

One potential source of performance benchmarks: [https://machinelearningmastery.com/binary-classification-tutorial-with-the-keras-deep-learning-library/]

Any deep-learning modeling project genrally can be broken down into about six major tasks:
0. Prepare Environment
1. Load Data
2. Define Model
3. Fit and Evaluate Model
4. Optimize Model
5. Finalize Model

# Section 0. Prepare Environment

In [1]:
# Create the random seed numbers for reproducible results
seedNum = 88

In [2]:
# Load libraries and packages
import random
random.seed(seedNum)
import numpy as np
np.random.seed(seedNum)
import tensorflow as tf
tf.set_random_seed(seedNum)
import pandas as pd
import os
import smtplib
from datetime import datetime
from email.message import EmailMessage
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [3]:
# Configure a new global `tensorflow` session
import keras as K
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.backend.set_session(sess)

Using TensorFlow backend.


In [4]:
# Begin the timer for the script processing
startTimeScript = datetime.now()

# Set up the flag to stop sending progress emails (setting to True will send status emails!)
notifyStatus = False

# Set the flag for splitting the dataset
splitDataset = True
splitPercentage = 0.25

# Set various default Keras modeling parameters
default_kernel_init = K.initializers.RandomNormal(seed=seedNum)
default_loss = 'binary_crossentropy'
default_optimizer = 'adam'
default_epochs = 100
default_batches = 5
default_metrics = ['accuracy']

# Set the number of folds for cross validation
folds = 10

In [5]:
# Set up the email notification function
def email_notify(msg_text):
    sender = os.environ.get('MAIL_SENDER')
    receiver = os.environ.get('MAIL_RECEIVER')
    gateway = os.environ.get('SMTP_GATEWAY')
    smtpuser = os.environ.get('SMTP_USERNAME')
    password = os.environ.get('SMTP_PASSWORD')
    if sender==None or receiver==None or gateway==None or smtpuser==None or password==None:
        sys.exit("Incomplete email setup info. Script Processing Aborted!!!")
    msg = EmailMessage()
    msg.set_content(msg_text)
    msg['Subject'] = 'Notification from Keras Binary Classification Script'
    msg['From'] = sender
    msg['To'] = receiver
    server = smtplib.SMTP(gateway, 587)
    server.starttls()
    server.login(smtpuser, password)
    server.send_message(msg)
    server.quit()

In [6]:
if (notifyStatus): email_notify("Phase 0 Prepare Environment completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 1. Load Data

In [7]:
if (notifyStatus): email_notify("Phase 1 Load Data has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [8]:
# Load the dataset
df_original = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data", header=None)
dataset = df_original.values
print(dataset)

[[0.02 0.0371 0.0428 ... 0.009 0.0032 'R']
 [0.0453 0.0523 0.0843 ... 0.0052 0.0044 'R']
 [0.0262 0.0582 0.1099 ... 0.0095 0.0078 'R']
 ...
 [0.0522 0.0437 0.018 ... 0.0077 0.0031 'M']
 [0.0303 0.0353 0.049 ... 0.0036 0.0048 'M']
 [0.026 0.0363 0.0136 ... 0.0061 0.0115 'M']]


In [9]:
# Split the original dataset into input (X) and output (y) variables
X_original = dataset[:,0:60].astype(float)
y_original = dataset[:,60]
print('Shape of X_original:', X_original.shape, '| Shape of y_original:', y_original.shape)

Shape of X_original: (208, 60) | Shape of y_original: (208,)


In [10]:
# Encode class values as integers and perform one-hot-encoding
encoder = LabelEncoder()
encoder.fit(y_original)
y_encoded = encoder.transform(y_original)
print(y_encoded)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [11]:
# Split the data further into training and test datasets
if (splitDataset):
    X_train, X_test, y_train, y_test = train_test_split(X_original, y_encoded, test_size=splitPercentage, random_state=seedNum)
else:
    X_train, y_train = X_original, y_encoded
    X_test, y_test = X_original, y_encoded
print('Shape of X_train:', X_train.shape, '| Shape of y_train:', y_train.shape)
print('Shape of X_test:', X_test.shape, '| Shape of y_test:', y_test.shape)

Shape of X_train: (156, 60) | Shape of y_train: (156,)
Shape of X_test: (52, 60) | Shape of y_test: (52,)


In [12]:
if (notifyStatus): email_notify("Phase 1 Load Data completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 2. Define Model

In [13]:
if (notifyStatus): email_notify("Phase 2 Define Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [14]:
# Define the Keras model required for KerasClassifier
def create_default_model():
    default_model = K.models.Sequential()
    default_model.add(Dense(60, input_dim=60, kernel_initializer=default_kernel_init, activation='relu'))
    default_model.add(Dense(1, kernel_initializer=default_kernel_init, activation='sigmoid'))
    default_model.compile(loss=default_loss, optimizer=default_optimizer, metrics=default_metrics)
    return default_model

In [15]:
# Initialize the Keras model
cv_model = KerasClassifier(build_fn=create_default_model, epochs=default_epochs, batch_size=default_batches, verbose=0)

In [16]:
if (notifyStatus): email_notify("Phase 2 Define Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 3. Fit and Evaluate Model

In [17]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [18]:
# Fit and evaluate the Keras model using 10-fold cross validation
kfold = StratifiedKFold(n_splits=folds, shuffle=True, random_state=seedNum)
results = cross_val_score(cv_model, X_train, y_train, cv=kfold)
print('Generating results using the metrics of', default_metrics)
print('All cross-Validate results:', results)
print('Baseline results [mean (std)]: %.2f%% (%.2f%%)' % (results.mean()*100, results.std()*100))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Generating results using the metrics of ['accuracy']
All cross-Validate results: [0.70588237 0.81250001 0.87500001 0.68750001 0.75000001 0.93333334
 0.73333335 0.80000001 0.60000001 0.93333334]
Baseline results [mean (std)]: 78.31% (10.34%)


In [19]:
if (notifyStatus): email_notify("Phase 3 Fit and Evaluate Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 4. Optimize Model

In [20]:
if (notifyStatus): email_notify("Phase 4 Optimize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [21]:
# Define the Keras model required for KerasClassifier
def create_customized_model(optimizer, kernel_init):
    customized_model = K.models.Sequential()
    customized_model.add(Dense(60, input_dim=60, kernel_initializer=kernel_init, activation='relu'))
    customized_model.add(Dense(1, kernel_initializer=kernel_init, activation='sigmoid'))
    customized_model.compile(loss=default_loss, optimizer=optimizer, metrics=default_metrics)
    return customized_model

In [22]:
# Create model
grid_model = KerasClassifier(build_fn=create_customized_model, verbose=0)

# Perform grid search using different epochs, batch sizes, and optimizers
optimizer_grid = ['rmsprop', 'adam']
init_grid = ['Constant', 'RandomNormal', 'RandomUniform']
epoch_grid = [100, 150, 200]
batch_grid = [5, 10, 15]
param_grid = dict(optimizer=optimizer_grid, kernel_init=init_grid, epochs=epoch_grid, batch_size=batch_grid)
grid = GridSearchCV(estimator=grid_model, param_grid=param_grid, cv=5, n_jobs=-1)
grid_result = grid.fit(X_train, y_train)

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
	print("%f (%f) with: %r" % (mean, stdev, param))

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:813: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best: 0.794872 using {'batch_size': 5, 'epochs': 200, 'kernel_init': 'RandomNormal', 'optimizer': 'adam'}
0.544872 (0.054248) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'Constant', 'optimizer': 'rmsprop'}
0.544872 (0.054248) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'Constant', 'optimizer': 'adam'}
0.762821 (0.067714) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomNormal', 'optimizer': 'rmsprop'}
0.788462 (0.060694) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomNormal', 'optimizer': 'adam'}
0.756410 (0.079483) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomUniform', 'optimizer': 'rmsprop'}
0.769231 (0.075656) with: {'batch_size': 5, 'epochs': 100, 'kernel_init': 'RandomUniform', 'optimizer': 'adam'}
0.544872 (0.054248) with: {'batch_size': 5, 'epochs': 150, 'kernel_init': 'Constant', 'optimizer': 'rmsprop'}
0.544872 (0.054248) with: {'batch_size': 5, 'epochs': 150, 'kernel_init': 'Constant', 'optimizer': 'adam'}
0.775641

In [23]:
best_optimizer = 'rmsprop'
best_kernel_init = 'RandomUniform'
best_epoch = 200
best_batch = 10

In [24]:
if (notifyStatus): email_notify("Phase 4 Optimize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

# Section 5. Finalize Model

In [25]:
if (notifyStatus): email_notify("Phase 5 Finalize Model has begun! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [26]:
# Create the final model for evaluating the test dataset
final_model = create_customized_model(best_optimizer, best_kernel_init)

In [27]:
final_model.fit(X_train, y_train, epochs=best_epoch, batch_size=best_batch, verbose=1)

Epoch 1/200
156/156 [==============================] - 2s 12ms/step - loss: 0.6908 - acc: 0.5128
Epoch 2/200
156/156 [==============================] - 0s 417us/step - loss: 0.6839 - acc: 0.5449
Epoch 3/200
156/156 [==============================] - 0s 414us/step - loss: 0.6789 - acc: 0.5449
Epoch 4/200
156/156 [==============================] - 0s 421us/step - loss: 0.6734 - acc: 0.5449
Epoch 5/200
156/156 [==============================] - 0s 697us/step - loss: 0.6680 - acc: 0.5513
Epoch 6/200
156/156 [==============================] - 0s 421us/step - loss: 0.6619 - acc: 0.5962
Epoch 7/200
156/156 [==============================] - 0s 417us/step - loss: 0.6529 - acc: 0.6282
Epoch 8/200
156/156 [==============================] - 0s 432us/step - loss: 0.6430 - acc: 0.6667
Epoch 9/200
156/156 [==============================] - 0s 440us/step - loss: 0.6370 - acc: 0.6795
Epoch 10/200
156/156 [==============================] - 0s 417us/step - loss: 0.6250 - acc: 0.6603
Epoch 11/200
156/156

In [28]:
# Display a summary of the final model
print(final_model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 60)                3660      
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 61        
Total params: 3,721
Trainable params: 3,721
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
# Evaluate the Keras model on previously unseen data
scores = final_model.evaluate(X_test, y_test)
print("\n%s: %.2f%%" % (final_model.metrics_names[1], scores[1]*100))
print("\n%s: %.2f%%" % (final_model.metrics_names[0], scores[0]*100))

52/52 [==============================] - 1s 11ms/step

acc: 82.69%

loss: 45.38%


In [30]:
# Make class predictions with the model
predictions = final_model.predict_classes(X_original)

# Summarize the first 20 cases
for i in range(20):
	print('%s => %d (expected %d)' % (X_test[i].tolist(), predictions[i], y_test[i]))

[0.0365, 0.1632, 0.1636, 0.1421, 0.113, 0.1306, 0.2112, 0.2268, 0.2992, 0.3735, 0.3042, 0.0387, 0.2679, 0.5397, 0.6204, 0.7257, 0.835, 0.6888, 0.445, 0.3921, 0.5605, 0.7545, 0.8311, 1.0, 0.8762, 0.7092, 0.7009, 0.5014, 0.3942, 0.4456, 0.4072, 0.0773, 0.1423, 0.0401, 0.3597, 0.6847, 0.7076, 0.3597, 0.0612, 0.3027, 0.3966, 0.3868, 0.238, 0.2059, 0.2288, 0.1704, 0.1587, 0.1792, 0.1022, 0.0151, 0.0223, 0.011, 0.0071, 0.0205, 0.0164, 0.0063, 0.0078, 0.0094, 0.011, 0.0068] => 1 (expected 1)
[0.0099, 0.0484, 0.0299, 0.0297, 0.0652, 0.1077, 0.2363, 0.2385, 0.0075, 0.1882, 0.1456, 0.1892, 0.3176, 0.134, 0.2169, 0.2458, 0.2589, 0.2786, 0.2298, 0.0656, 0.1441, 0.1179, 0.1668, 0.1783, 0.2476, 0.257, 0.1036, 0.5356, 0.7124, 0.6291, 0.4756, 0.6015, 0.7208, 0.6234, 0.5725, 0.7523, 0.8712, 0.9252, 0.9709, 0.9297, 0.8995, 0.7911, 0.56, 0.2838, 0.4407, 0.5507, 0.4331, 0.2905, 0.1981, 0.0779, 0.0396, 0.0173, 0.0149, 0.0115, 0.0202, 0.0139, 0.0029, 0.016, 0.0106, 0.0134] => 1 (expected 1)
[0.0308, 0.0339,

In [31]:
if (notifyStatus): email_notify("Phase 5 Finalize Model completed! "+datetime.now().strftime('%a %B %d, %Y %I:%M:%S %p'))

In [32]:
print ('Total time for the script:',(datetime.now() - startTimeScript))

Total time for the script: 0:21:38.562817
